# Example Large Language Model (LLM)

This notebook demonstrates how to use the [Meta-Llama-3.1-8B-Instruct](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct) with [TensorRT-LLM](https://github.com/NVIDIA/TensorRT-LLM) and the `TextGenerator` class (which inherits from `BaseModel`).

The 1st step is to download the `huggingface` checkpoint :
```python
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = 'meta-llama/Meta-Llama-3.1-8B-Instruct'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model     = AutoModelForCausalLM.from_pretrained(model_name, device_map = 'cuda', torch_dtype = 'float16')
```

Make sure to set the `HF_TOKEN` environment variable with your specific token, and request the permission to use this specific model. If you want to use another model, the procedure is exactly the same ! Simply refer to the appropriate example file in the `TensorRT-LLM/examples` file to convert the checkpoint ;)

Once the model is loaded, let's create the `TensorRT-LLM` checkpoint + runtime engine (the below commands come from the official `TensorRT-LLM/examples/llama` readme file, do not forget to adapt them if you use another architecture).
```bash
git clone https://github.com/NVIDIA/TensorRT-LLM
cd TensorRT-LLM/examples/llama

# Convert checkpoints
# Make sure to adapt the path to the huggingface snapshots
python convert_checkpoint.py --model_dir ~/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/snapshots/<directory>/ --output_dir llama-3.1-8B-Instruc-checkpointt --dtype float16 --tp_size 1
# You can add these arguments to build an int-8 inference model
# --use_weight_only --weight_only_precision int8

# Now build the engine
trtllm-build --checkpoint_dir llama-3.1-8B-Instruc-checkpointt --output_dir llama-3.1-8B-Instruc-engine --gemm_plugin auto
```

Finally, the `TextGenerator` class can be instanciated by passing the `restoration_mode = 'trt_llm'` to use `TensorRT-LLM` runner, and the `directory` argument to point to the `TRT-LLM` engine.

The following cells demonstrate how to use the `TextGenerator` class to perform various `Natural Language Understanding (NLU)` tasks, either with pre-defined prompts, either by providing custom ones !

**Important Note** : this notebook is **experimental**, and is a first implementation of `TensorRT-LLM`-based inference. The codebase will largely evolves in the future, including new features, such as streaming, logits processing, output formatting, function calling, ...

In [1]:
import os
import json
import keras
import importlib
import numpy as np

from utils import *
from loggers import set_level
from utils.keras_utils import ops
from utils.text import text_encoder
from models.nlu import TextGenerator

model = TextGenerator(
    lang = 'multi',
    path = 'pretrained_models/pretrained_weights/llama-3.1-8B-Instruct-engine',
    runtime = 'trt_llm',
    text_encoder = 'meta-llama/Meta-Llama-3.1-8B-Instruct',
    name = 'trt_llm_llama-3.1_8B_instruct',
    save = False
)

2024-12-31 08:09:11.213178: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-31 08:09:11.222456: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735628951.231829    2236 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735628951.234559    2236 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-31 08:09:11.257216: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

[TensorRT-LLM] TensorRT-LLM version: 0.15.0
[TensorRT-LLM][INFO] Engine version 0.15.0 found in the config file, assuming engine(s) built by new builder API.
[TensorRT-LLM][INFO] MPI size: 1, MPI local size: 1, rank: 0
[TensorRT-LLM][INFO] Engine version 0.15.0 found in the config file, assuming engine(s) built by new builder API.
[TensorRT-LLM][INFO] Refreshed the MPI local session
[TensorRT-LLM][INFO] MPI size: 1, MPI local size: 1, rank: 0
[TensorRT-LLM][INFO] Rank 0 is using GPU 0
[TensorRT-LLM][INFO] TRTGptModel maxNumSequences: 8
[TensorRT-LLM][INFO] TRTGptModel maxBatchSize: 8
[TensorRT-LLM][INFO] TRTGptModel maxBeamWidth: 1
[TensorRT-LLM][INFO] TRTGptModel maxSequenceLen: 24576
[TensorRT-LLM][INFO] TRTGptModel maxDraftLen: 0
[TensorRT-LLM][INFO] TRTGptModel mMaxAttentionWindowSize: (24576) * 32
[TensorRT-LLM][INFO] TRTGptModel enableTrtOverlap: 0
[TensorRT-LLM][INFO] TRTGptModel normalizeLogProbs: 1
[TensorRT-LLM][INFO] TRTGptModel maxNumTokens: 16384
[TensorRT-LLM][INFO] TRTGp

In [2]:
print(model.chat_template)

{{- bos_token }}
{%- if custom_tools is defined %}
    {%- set tools = custom_tools %}
{%- endif %}
{%- if not tools_in_user_message is defined %}
    {%- set tools_in_user_message = true %}
{%- endif %}
{%- if not date_string is defined %}
    {%- set date_string = "26 Jul 2024" %}
{%- endif %}
{%- if not tools is defined %}
    {%- set tools = none %}
{%- endif %}

{#- This block extracts the system message, so we can slot it into the right place. #}
{%- if messages[0]['role'] == 'system' %}
    {%- set system_message = messages[0]['content']|trim %}
    {%- set messages = messages[1:] %}
{%- else %}
    {%- set system_message = "" %}
{%- endif %}

{#- System message + builtin tools #}
{{- "<|start_header_id|>system<|end_header_id|>\n\n" }}
{%- if builtin_tools is defined or tools is not none %}
    {{- "Environment: ipython\n" }}
{%- endif %}
{%- if builtin_tools is defined %}
    {{- "Tools: " + builtin_tools | reject('equalto', 'code_interpreter') | join(", ") + "\n\n"}}
{%- endif

### Open-domain question-answering (ODQA)

The `Open-domain question-answering (ODQA)` task is the most generic one, where the objective is to answer general questions with general answers. All the following downstream tasks can actually be executed as ODQA with a specific question. This is exactly what the additional functions do : adding a specific prompt dedicated to the task (this will be shown in the next task demo).

In [2]:
text = "Hello World !"

res = model.answer(text)[0]

print('# input tokens : {}\nInput :\n{}Answer :\n{}'.format(
    len(res['input_tokens']), res['input'], res['content']
))

47 tokens generated in 1.062 sec (44.235 tokens/sec)
# input tokens : 57
Input :
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 31 December 2024

You are an AI assistant. You have to answer the best as possible to the user's queries<|start_header_id|>user<|end_header_id|>

Hello World !<|start_header_id|>assistant<|end_header_id|>

Answer :
Hello World! It's great to see you here. How can I assist you today? Do you have any questions, need help with a task, or just want to chat? I'm all ears (or rather, all text)!


In [4]:
text = "Bonjour !"

res = model.answer(text, lang = 'fr')[0]

print('# input tokens : {}\nInput :\n{}Answer :\n{}'.format(
    len(res['input_tokens']), res['input'], res['content']
))

11 tokens generated in 213 ms (51.569 tokens/sec)
# input tokens : 60
Input :
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 31 December 2024

Tu es un assistant IA. Tu dois répondre de manière brève et précise aux questions de l'utilisateur.<|start_header_id|>user<|end_header_id|>

Bonjour !<|start_header_id|>assistant<|end_header_id|>

Answer :
Bonjour ! Comment puis-je vous aider aujourd'hui ?


### Translation

In this case, the `translate` method adds a specific prompt to specify the target language.

This is shown in the `input` field : `"Translate this text in French :"`. In the second cell, it is shown how to override this behavior by specifying a custom prompt. In this second case, it is actually not required to call the custom `translate` method, as the additional prompt is overriden : calling the `answer` method would have the exact same result.

In [5]:
text = """
The `Open-domain question-answering (ODQA)` task is the most generic one, where the objective is to answer \
general questions with general answers. All the following downstream tasks can actually be executed as ODQA \
with a specific question. This is exactly what the additional functions do : adding a specific prompt dedicated to \
the task (this will be shown in the next task demo).
""".strip()

res = model.translate(text, lang = 'en')[0]

print('# input tokens : {}\nInput :\n{}Answer :\n{}'.format(
    len(res['input_tokens']), res['input'], res['content']
))

121 tokens generated in 2.132 sec (56.765 tokens/sec)
# input tokens : 143
Input :
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 31 December 2024

You are an AI assistant. You have to answer the best as possible to the user's queries<|start_header_id|>user<|end_header_id|>

Translate this text in French :

{text}The `Open-domain question-answering (ODQA)` task is the most generic one, where the objective is to answer general questions with general answers. All the following downstream tasks can actually be executed as ODQA with a specific question. This is exactly what the additional functions do : adding a specific prompt dedicated to the task (this will be shown in the next task demo).<|start_header_id|>assistant<|end_header_id|>

The translation is :

Answer :
The translation is :

La tâche de `question-réponse à domaine ouvert (ODQA)` est la plus générique, où l'objectif est de répondre à des questions générales avec 

In [6]:
text = """
The `Open-domain question-answering (ODQA)` task is the most generic one, where the objective is to answer \
general questions with general answers. All the following downstream tasks can actually be executed as ODQA \
with a specific question. This is exactly what the additional functions do : adding a specific prompt dedicated to \
the task (this will be shown in the next task demo).
""".strip()

res = model.translate(text, format = 'Translate this text in Spanish :\n\n{text}')[0]

print('# input tokens : {}\nInput :\n{}Answer :\n{}'.format(
    len(res['input_tokens']), res['input'], res['content']
))

103 tokens generated in 1.821 sec (56.550 tokens/sec)
# input tokens : 141
Input :
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 31 December 2024

You are an AI assistant. You have to answer the best as possible to the user's queries<|start_header_id|>user<|end_header_id|>

Translate this text in Spanish :

The `Open-domain question-answering (ODQA)` task is the most generic one, where the objective is to answer general questions with general answers. All the following downstream tasks can actually be executed as ODQA with a specific question. This is exactly what the additional functions do : adding a specific prompt dedicated to the task (this will be shown in the next task demo).<|start_header_id|>assistant<|end_header_id|>

The translation is :

Answer :
The translation is :

La tarea de `preguntas y respuestas de dominio abierto (ODQA)` es la más genérica, donde el objetivo es responder preguntas generales con respue

### Text reformulation

Most of the time, a 1st translation attempt is not perfect (most probably because it's an 8B model), and may have fluency / style / grammar issues. The default reformulation prompt aims to improve this though a dedicated prompt. 

In [7]:
text = """
La tâche de question-réponse à domaine ouvert (ODQA) est la plus générique, où l'objectif est de répondre à des questions \
générales avec des réponses générales. Toutes les tâches downstream qui suivent peuvent en réalité être exécutées comme \
ODQA avec une question spécifique. C'est exactement ce que font les fonctions supplémentaires : ajouter un prompt \
spécifique dédié à la tâche (cela sera montré dans la prochaine démo de tâche).
""".strip()

res = model.reformulate(text, lang = 'fr')[0]

print('# input tokens : {}\nInput :\n{}Answer :\n{}'.format(
    len(res['input_tokens']), res['input'], res['content']
))

111 tokens generated in 1.962 sec (56.576 tokens/sec)
# input tokens : 209
Input :
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 31 December 2024

Tu es un assistant IA. Tu dois répondre de manière brève et précise aux questions de l'utilisateur.<|start_header_id|>user<|end_header_id|>

Réécris ce texte **sans expliquer les changements** en améliorant le style, la grammaire et la fluidité :

La tâche de question-réponse à domaine ouvert (ODQA) est la plus générique, où l'objectif est de répondre à des questions générales avec des réponses générales. Toutes les tâches downstream qui suivent peuvent en réalité être exécutées comme ODQA avec une question spécifique. C'est exactement ce que font les fonctions supplémentaires : ajouter un prompt spécifique dédié à la tâche (cela sera montré dans la prochaine démo de tâche).<|start_header_id|>assistant<|end_header_id|>

Answer :
La tâche de question-réponse à domaine ouvert (OD

### Text summarization

The text comes from [this wikipedia article](https://en.wikipedia.org/wiki/Question_answering) about Q&A. The model has generated 241 tokens based on 514 tokens (wikipedia article + input prompt representing a bit less than 100 tokens), 

In [2]:
text = """
Question-answering research attempts to develop ways of answering a wide range of question types, including fact, list, definition, how, why, hypothetical, semantically constrained, and cross-lingual questions.
- Answering questions related to an article in order to evaluate reading comprehension is one of the simpler form of question answering, since a given article is relatively short compared to the domains of other types of question-answering problems. An example of such a question is "What did Albert Einstein win the Nobel Prize for?" after an article about this subject is given to the system.
- Closed-book question answering is when a system has memorized some facts during training and can answer questions without explicitly being given a context. This is similar to humans taking closed-book exams.
- Closed-domain question answering deals with questions under a specific domain (for example, medicine or automotive maintenance) and can exploit domain-specific knowledge frequently formalized in ontologies. Alternatively, "closed-domain" might refer to a situation where only a limited type of questions are accepted, such as questions asking for descriptive rather than procedural information. Question answering systems in the context of[vague] machine reading applications have also been constructed in the medical domain, for instance related to[vague] Alzheimer's disease.[3]
- Open-domain question answering deals with questions about nearly anything and can only rely on general ontologies and world knowledge. Systems designed for open-domain question answering usually have much more data available from which to extract the answer. An example of an open-domain question is "What did Albert Einstein win the Nobel Prize for?" while no article about this subject is given to the system.

Another way to categorize question-answering systems is by the technical approach used. There are a number of different types of QA systems, including
- rule-based systems,
- statistical systems, and
- hybrid systems.

Rule-based systems use a set of rules to determine the correct answer to a question. Statistical systems use statistical methods to find the most likely answer to a question. Hybrid systems use a combination of rule-based and statistical methods.
""".strip()

res = model.summarize(text, lang = 'en', max_input_len = 4096, max_new_tokens = 4096)[0]

print('# input tokens : {}\nInput :\n{}Answer :\n{}'.format(
    len(res['input_tokens']), res['input'], res['content']
))

251 tokens generated in 4.655 sec (53.916 tokens/sec)
# input tokens : 491
Input :
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 31 December 2024

You are an AI assistant. You have to answer the best as possible to the user's queries<|start_header_id|>user<|end_header_id|>

Write a short and concise summary of this text while keeping all the important information :

Question-answering research attempts to develop ways of answering a wide range of question types, including fact, list, definition, how, why, hypothetical, semantically constrained, and cross-lingual questions.
- Answering questions related to an article in order to evaluate reading comprehension is one of the simpler form of question answering, since a given article is relatively short compared to the domains of other types of question-answering problems. An example of such a question is "What did Albert Einstein win the Nobel Prize for?" after an article abo

### Entity extraction

Instead of summarizing a text, we may want to have a structured result of some key information from the text. Here is a more complex example of entity extraction with `jinja` text format.

The text comes from [this huggingface](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct) page about LLaMA-3.1 8B Instruct (the full text).

This is the prompt used to format this query :
```python
# This represents a simple string
{{- "Find the requested information in the below text, and add them in the JSON structure. If an information is missing, set the value `null`.\n\n" }}
# This adds the provided input text
{{- text + "\n\n" }}

{{- "Here is the requested information : {\n" }}
# This iterates over the provided entities (a `dict`-like) to add them
{%- for key, value in entities.items() %}
    {{- "    '" + key + "' : # " + value + "\n" }}
{%- endfor %}
{{- "}\n" }}
```

In [4]:
text = """
 Model Information

The Meta Llama 3.1 collection of multilingual large language models (LLMs) is a collection of pretrained and instruction tuned generative models in 8B, 70B and 405B sizes (text in/text out). The Llama 3.1 instruction tuned text only models (8B, 70B, 405B) are optimized for multilingual dialogue use cases and outperform many of the available open source and closed chat models on common industry benchmarks.

Model developer: Meta

Model Architecture: Llama 3.1 is an auto-regressive language model that uses an optimized transformer architecture. The tuned versions use supervised fine-tuning (SFT) and reinforcement learning with human feedback (RLHF) to align with human preferences for helpfulness and safety.
	Training Data 	Params 	Input modalities 	Output modalities 	Context length 	GQA 	Token count 	Knowledge cutoff
Llama 3.1 (text only) 	A new mix of publicly available online data. 	8B 	Multilingual Text 	Multilingual Text and code 	128k 	Yes 	15T+ 	December 2023
70B 	Multilingual Text 	Multilingual Text and code 	128k 	Yes
405B 	Multilingual Text 	Multilingual Text and code 	128k 	Yes

Supported languages: English, German, French, Italian, Portuguese, Hindi, Spanish, and Thai.

Llama 3.1 family of models. Token counts refer to pretraining data only. All model versions use Grouped-Query Attention (GQA) for improved inference scalability.

Model Release Date: July 23, 2024.

Status: This is a static model trained on an offline dataset. Future versions of the tuned models will be released as we improve model safety with community feedback.

License: A custom commercial license, the Llama 3.1 Community License, is available at: https://github.com/meta-llama/llama-models/blob/main/models/llama3_1/LICENSE

Where to send questions or comments about the model Instructions on how to provide feedback or comments on the model can be found in the model README. For more technical information about generation parameters and recipes for how to use Llama 3.1 in applications, please go here.
Intended Use

Intended Use Cases Llama 3.1 is intended for commercial and research use in multiple languages. Instruction tuned text only models are intended for assistant-like chat, whereas pretrained models can be adapted for a variety of natural language generation tasks. The Llama 3.1 model collection also supports the ability to leverage the outputs of its models to improve other models including synthetic data generation and distillation. The Llama 3.1 Community License allows for these use cases.

Out-of-scope Use in any manner that violates applicable laws or regulations (including trade compliance laws). Use in any other way that is prohibited by the Acceptable Use Policy and Llama 3.1 Community License. Use in languages beyond those explicitly referenced as supported in this model card**.

**Note: Llama 3.1 has been trained on a broader collection of languages than the 8 supported languages. Developers may fine-tune Llama 3.1 models for languages beyond the 8 supported languages provided they comply with the Llama 3.1 Community License and the Acceptable Use Policy and in such cases are responsible for ensuring that any uses of Llama 3.1 in additional languages is done in a safe and responsible manner.
""".strip()

text = """
 Model Information

The Meta Llama 3.1 collection of multilingual large language models (LLMs) is a collection of pretrained and instruction tuned generative models in 8B, 70B and 405B sizes (text in/text out). The Llama 3.1 instruction tuned text only models (8B, 70B, 405B) are optimized for multilingual dialogue use cases and outperform many of the available open source and closed chat models on common industry benchmarks.

Model developer: Meta

Model Architecture: Llama 3.1 is an auto-regressive language model that uses an optimized transformer architecture. The tuned versions use supervised fine-tuning (SFT) and reinforcement learning with human feedback (RLHF) to align with human preferences for helpfulness and safety.
	Training Data 	Params 	Input modalities 	Output modalities 	Context length 	GQA 	Token count 	Knowledge cutoff
Llama 3.1 (text only) 	A new mix of publicly available online data. 	8B 	Multilingual Text 	Multilingual Text and code 	128k 	Yes 	15T+ 	December 2023
70B 	Multilingual Text 	Multilingual Text and code 	128k 	Yes
405B 	Multilingual Text 	Multilingual Text and code 	128k 	Yes

Supported languages: English, German, French, Italian, Portuguese, Hindi, Spanish, and Thai.

Llama 3.1 family of models. Token counts refer to pretraining data only. All model versions use Grouped-Query Attention (GQA) for improved inference scalability.

Model Release Date: July 23, 2024.

Status: This is a static model trained on an offline dataset. Future versions of the tuned models will be released as we improve model safety with community feedback.

License: A custom commercial license, the Llama 3.1 Community License, is available at: https://github.com/meta-llama/llama-models/blob/main/models/llama3_1/LICENSE

Where to send questions or comments about the model Instructions on how to provide feedback or comments on the model can be found in the model README. For more technical information about generation parameters and recipes for how to use Llama 3.1 in applications, please go here.
Intended Use

Intended Use Cases Llama 3.1 is intended for commercial and research use in multiple languages. Instruction tuned text only models are intended for assistant-like chat, whereas pretrained models can be adapted for a variety of natural language generation tasks. The Llama 3.1 model collection also supports the ability to leverage the outputs of its models to improve other models including synthetic data generation and distillation. The Llama 3.1 Community License allows for these use cases.

Out-of-scope Use in any manner that violates applicable laws or regulations (including trade compliance laws). Use in any other way that is prohibited by the Acceptable Use Policy and Llama 3.1 Community License. Use in languages beyond those explicitly referenced as supported in this model card**.

**Note: Llama 3.1 has been trained on a broader collection of languages than the 8 supported languages. Developers may fine-tune Llama 3.1 models for languages beyond the 8 supported languages provided they comply with the Llama 3.1 Community License and the Acceptable Use Policy and in such cases are responsible for ensuring that any uses of Llama 3.1 in additional languages is done in a safe and responsible manner.
How to use

This repository contains two versions of Meta-Llama-3.1-8B-Instruct, for use with transformers and with the original llama codebase.
Use with transformers

Starting with transformers >= 4.43.0 onward, you can run conversational inference using the Transformers pipeline abstraction or by leveraging the Auto classes with the generate() function.

Make sure to update your transformers installation via pip install --upgrade transformers.

import transformers
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

Note: You can also find detailed recipes on how to use the model locally, with torch.compile(), assisted generations, quantised and more at huggingface-llama-recipes
Tool use with transformers

LLaMA-3.1 supports multiple tool use formats. You can see a full guide to prompt formatting here.

Tool use is also supported through chat templates in Transformers. Here is a quick example showing a single simple tool:

# First, define a tool
def get_current_temperature(location: str) -> float:
    \"""
    Get the current temperature at a location.
    
    Args:
        location: The location to get the temperature for, in the format "City, Country"
    Returns:
        The current temperature at the specified location in the specified units, as a float.
    \"""
    return 22.  # A real function should probably actually get the temperature!

# Next, create a chat and apply the chat template
messages = [
  {"role": "system", "content": "You are a bot that responds to weather queries."},
  {"role": "user", "content": "Hey, what's the temperature in Paris right now?"}
]

inputs = tokenizer.apply_chat_template(messages, tools=[get_current_temperature], add_generation_prompt=True)

You can then generate text from this input as normal. If the model generates a tool call, you should add it to the chat like so:

tool_call = {"name": "get_current_temperature", "arguments": {"location": "Paris, France"}}
messages.append({"role": "assistant", "tool_calls": [{"type": "function", "function": tool_call}]})

and then call the tool and append the result, with the tool role, like so:

messages.append({"role": "tool", "name": "get_current_temperature", "content": "22.0"})

After that, you can generate() again to let the model use the tool result in the chat. Note that this was a very brief introduction to tool calling - for more information, see the LLaMA prompt format docs and the Transformers tool use documentation.
Use with llama

Please, follow the instructions in the repository

To download Original checkpoints, see the example command below leveraging huggingface-cli:

huggingface-cli download meta-llama/Meta-Llama-3.1-8B-Instruct --include "original/*" --local-dir Meta-Llama-3.1-8B-Instruct

Hardware and Software

Training Factors We used custom training libraries, Meta's custom built GPU cluster, and production infrastructure for pretraining. Fine-tuning, annotation, and evaluation were also performed on production infrastructure.

Training utilized a cumulative of 39.3M GPU hours of computation on H100-80GB (TDP of 700W) type hardware, per the table below. Training time is the total GPU time required for training each model and power consumption is the peak power capacity per GPU device used, adjusted for power usage efficiency.

Training Greenhouse Gas Emissions Estimated total location-based greenhouse gas emissions were 11,390 tons CO2eq for training. Since 2020, Meta has maintained net zero greenhouse gas emissions in its global operations and matched 100% of its electricity use with renewable energy, therefore the total market-based greenhouse gas emissions for training were 0 tons CO2eq.
	Training Time (GPU hours) 	Training Power Consumption (W) 	Training Location-Based Greenhouse Gas Emissions

(tons CO2eq)
	Training Market-Based Greenhouse Gas Emissions

(tons CO2eq)
Llama 3.1 8B 	1.46M 	700 	420 	0
Llama 3.1 70B 	7.0M 	700 	2,040 	0
Llama 3.1 405B 	30.84M 	700 	8,930 	0
Total 	39.3M 	

	11,390 	0

The methodology used to determine training energy use and greenhouse gas emissions can be found here. Since Meta is openly releasing these models, the training energy use and greenhouse gas emissions will not be incurred by others.
Training Data

Overview: Llama 3.1 was pretrained on ~15 trillion tokens of data from publicly available sources. The fine-tuning data includes publicly available instruction datasets, as well as over 25M synthetically generated examples.

Data Freshness: The pretraining data has a cutoff of December 2023.
Benchmark scores

In this section, we report the results for Llama 3.1 models on standard automatic benchmarks. For all the evaluations, we use our internal evaluations library.
Base pretrained models
Category 	Benchmark 	# Shots 	Metric 	Llama 3 8B 	Llama 3.1 8B 	Llama 3 70B 	Llama 3.1 70B 	Llama 3.1 405B
General 	MMLU 	5 	macro_avg/acc_char 	66.7 	66.7 	79.5 	79.3 	85.2
MMLU-Pro (CoT) 	5 	macro_avg/acc_char 	36.2 	37.1 	55.0 	53.8 	61.6
AGIEval English 	3-5 	average/acc_char 	47.1 	47.8 	63.0 	64.6 	71.6
CommonSenseQA 	7 	acc_char 	72.6 	75.0 	83.8 	84.1 	85.8
Winogrande 	5 	acc_char 	- 	60.5 	- 	83.3 	86.7
BIG-Bench Hard (CoT) 	3 	average/em 	61.1 	64.2 	81.3 	81.6 	85.9
ARC-Challenge 	25 	acc_char 	79.4 	79.7 	93.1 	92.9 	96.1
Knowledge reasoning 	TriviaQA-Wiki 	5 	em 	78.5 	77.6 	89.7 	89.8 	91.8
Reading comprehension 	SQuAD 	1 	em 	76.4 	77.0 	85.6 	81.8 	89.3
QuAC (F1) 	1 	f1 	44.4 	44.9 	51.1 	51.1 	53.6
BoolQ 	0 	acc_char 	75.7 	75.0 	79.0 	79.4 	80.0
DROP (F1) 	3 	f1 	58.4 	59.5 	79.7 	79.6 	84.8
Instruction tuned models
Category 	Benchmark 	# Shots 	Metric 	Llama 3 8B Instruct 	Llama 3.1 8B Instruct 	Llama 3 70B Instruct 	Llama 3.1 70B Instruct 	Llama 3.1 405B Instruct
General 	MMLU 	5 	macro_avg/acc 	68.5 	69.4 	82.0 	83.6 	87.3
MMLU (CoT) 	0 	macro_avg/acc 	65.3 	73.0 	80.9 	86.0 	88.6
MMLU-Pro (CoT) 	5 	micro_avg/acc_char 	45.5 	48.3 	63.4 	66.4 	73.3
IFEval 			76.8 	80.4 	82.9 	87.5 	88.6
Reasoning 	ARC-C 	0 	acc 	82.4 	83.4 	94.4 	94.8 	96.9
GPQA 	0 	em 	34.6 	30.4 	39.5 	46.7 	50.7
Code 	HumanEval 	0 	pass@1 	60.4 	72.6 	81.7 	80.5 	89.0
MBPP ++ base version 	0 	pass@1 	70.6 	72.8 	82.5 	86.0 	88.6
Multipl-E HumanEval 	0 	pass@1 	- 	50.8 	- 	65.5 	75.2
Multipl-E MBPP 	0 	pass@1 	- 	52.4 	- 	62.0 	65.7
Math 	GSM-8K (CoT) 	8 	em_maj1@1 	80.6 	84.5 	93.0 	95.1 	96.8
MATH (CoT) 	0 	final_em 	29.1 	51.9 	51.0 	68.0 	73.8
Tool Use 	API-Bank 	0 	acc 	48.3 	82.6 	85.1 	90.0 	92.0
BFCL 	0 	acc 	60.3 	76.1 	83.0 	84.8 	88.5
Gorilla Benchmark API Bench 	0 	acc 	1.7 	8.2 	14.7 	29.7 	35.3
Nexus (0-shot) 	0 	macro_avg/acc 	18.1 	38.5 	47.8 	56.7 	58.7
Multilingual 	Multilingual MGSM (CoT) 	0 	em 	- 	68.9 	- 	86.9 	91.6
Multilingual benchmarks
Category 	Benchmark 	Language 	Llama 3.1 8B 	Llama 3.1 70B 	Llama 3.1 405B
General 	MMLU (5-shot, macro_avg/acc) 	Portuguese 	62.12 	80.13 	84.95
Spanish 	62.45 	80.05 	85.08
Italian 	61.63 	80.4 	85.04
German 	60.59 	79.27 	84.36
French 	62.34 	79.82 	84.66
Hindi 	50.88 	74.52 	80.31
Thai 	50.32 	72.95 	78.21
Responsibility & Safety

As part of our Responsible release approach, we followed a three-pronged strategy to managing trust & safety risks:

    Enable developers to deploy helpful, safe and flexible experiences for their target audience and for the use cases supported by Llama.
    Protect developers against adversarial users aiming to exploit Llama capabilities to potentially cause harm.
    Provide protections for the community to help prevent the misuse of our models.

Responsible deployment

Llama is a foundational technology designed to be used in a variety of use cases, examples on how Meta’s Llama models have been responsibly deployed can be found in our Community Stories webpage. Our approach is to build the most helpful models enabling the world to benefit from the technology power, by aligning our model safety for the generic use cases addressing a standard set of harms. Developers are then in the driver seat to tailor safety for their use case, defining their own policy and deploying the models with the necessary safeguards in their Llama systems. Llama 3.1 was developed following the best practices outlined in our Responsible Use Guide, you can refer to the Responsible Use Guide to learn more.
Llama 3.1 instruct

Our main objectives for conducting safety fine-tuning are to provide the research community with a valuable resource for studying the robustness of safety fine-tuning, as well as to offer developers a readily available, safe, and powerful model for various applications to reduce the developer workload to deploy safe AI systems. For more details on the safety mitigations implemented please read the Llama 3 paper.

Fine-tuning data

We employ a multi-faceted approach to data collection, combining human-generated data from our vendors with synthetic data to mitigate potential safety risks. We’ve developed many large language model (LLM)-based classifiers that enable us to thoughtfully select high-quality prompts and responses, enhancing data quality control.

Refusals and Tone

Building on the work we started with Llama 3, we put a great emphasis on model refusals to benign prompts as well as refusal tone. We included both borderline and adversarial prompts in our safety data strategy, and modified our safety data responses to follow tone guidelines.
Llama 3.1 systems

Large language models, including Llama 3.1, are not designed to be deployed in isolation but instead should be deployed as part of an overall AI system with additional safety guardrails as required. Developers are expected to deploy system safeguards when building agentic systems. Safeguards are key to achieve the right helpfulness-safety alignment as well as mitigating safety and security risks inherent to the system and any integration of the model or system with external tools.

As part of our responsible release approach, we provide the community with safeguards that developers should deploy with Llama models or other LLMs, including Llama Guard 3, Prompt Guard and Code Shield. All our reference implementations demos contain these safeguards by default so developers can benefit from system-level safety out-of-the-box.
New capabilities

Note that this release introduces new capabilities, including a longer context window, multilingual inputs and outputs and possible integrations by developers with third party tools. Building with these new capabilities requires specific considerations in addition to the best practices that generally apply across all Generative AI use cases.

Tool-use: Just like in standard software development, developers are responsible for the integration of the LLM with the tools and services of their choice. They should define a clear policy for their use case and assess the integrity of the third party services they use to be aware of the safety and security limitations when using this capability. Refer to the Responsible Use Guide for best practices on the safe deployment of the third party safeguards.

Multilinguality: Llama 3.1 supports 7 languages in addition to English: French, German, Hindi, Italian, Portuguese, Spanish, and Thai. Llama may be able to output text in other languages than those that meet performance thresholds for safety and helpfulness. We strongly discourage developers from using this model to converse in non-supported languages without implementing finetuning and system controls in alignment with their policies and the best practices shared in the Responsible Use Guide.
Evaluations

We evaluated Llama models for common use cases as well as specific capabilities. Common use cases evaluations measure safety risks of systems for most commonly built applications including chat bot, coding assistant, tool calls. We built dedicated, adversarial evaluation datasets and evaluated systems composed of Llama models and Llama Guard 3 to filter input prompt and output response. It is important to evaluate applications in context, and we recommend building dedicated evaluation dataset for your use case. Prompt Guard and Code Shield are also available if relevant to the application.

Capability evaluations measure vulnerabilities of Llama models inherent to specific capabilities, for which were crafted dedicated benchmarks including long context, multilingual, tools calls, coding or memorization.

Red teaming

For both scenarios, we conducted recurring red teaming exercises with the goal of discovering risks via adversarial prompting and we used the learnings to improve our benchmarks and safety tuning datasets.

We partnered early with subject-matter experts in critical risk areas to understand the nature of these real-world harms and how such models may lead to unintended harm for society. Based on these conversations, we derived a set of adversarial goals for the red team to attempt to achieve, such as extracting harmful information or reprogramming the model to act in a potentially harmful capacity. The red team consisted of experts in cybersecurity, adversarial machine learning, responsible AI, and integrity in addition to multilingual content specialists with background in integrity issues in specific geographic markets.
Critical and other risks

We specifically focused our efforts on mitigating the following critical risk areas:

1- CBRNE (Chemical, Biological, Radiological, Nuclear, and Explosive materials) helpfulness

To assess risks related to proliferation of chemical and biological weapons, we performed uplift testing designed to assess whether use of Llama 3.1 models could meaningfully increase the capabilities of malicious actors to plan or carry out attacks using these types of weapons.

2. Child Safety

Child Safety risk assessments were conducted using a team of experts, to assess the model’s capability to produce outputs that could result in Child Safety risks and inform on any necessary and appropriate risk mitigations via fine tuning. We leveraged those expert red teaming sessions to expand the coverage of our evaluation benchmarks through Llama 3 model development. For Llama 3, we conducted new in-depth sessions using objective based methodologies to assess the model risks along multiple attack vectors including the additional languages Llama 3 is trained on. We also partnered with content specialists to perform red teaming exercises assessing potentially violating content while taking account of market specific nuances or experiences.

3. Cyber attack enablement

Our cyber attack uplift study investigated whether LLMs can enhance human capabilities in hacking tasks, both in terms of skill level and speed.

Our attack automation study focused on evaluating the capabilities of LLMs when used as autonomous agents in cyber offensive operations, specifically in the context of ransomware attacks. This evaluation was distinct from previous studies that considered LLMs as interactive assistants. The primary objective was to assess whether these models could effectively function as independent agents in executing complex cyber-attacks without human intervention.

Our study of Llama-3.1-405B’s social engineering uplift for cyber attackers was conducted to assess the effectiveness of AI models in aiding cyber threat actors in spear phishing campaigns. Please read our Llama 3.1 Cyber security whitepaper to learn more.
Community

Generative AI safety requires expertise and tooling, and we believe in the strength of the open community to accelerate its progress. We are active members of open consortiums, including the AI Alliance, Partnership on AI and MLCommons, actively contributing to safety standardization and transparency. We encourage the community to adopt taxonomies like the MLCommons Proof of Concept evaluation to facilitate collaboration and transparency on safety and content evaluations. Our Purple Llama tools are open sourced for the community to use and widely distributed across ecosystem partners including cloud service providers. We encourage community contributions to our Github repository.

We also set up the Llama Impact Grants program to identify and support the most compelling applications of Meta’s Llama model for societal benefit across three categories: education, climate and open innovation. The 20 finalists from the hundreds of applications can be found here.

Finally, we put in place a set of resources including an output reporting mechanism and bug bounty program to continuously improve the Llama technology with the help of the community.
Ethical Considerations and Limitations

The core values of Llama 3.1 are openness, inclusivity and helpfulness. It is meant to serve everyone, and to work for a wide range of use cases. It is thus designed to be accessible to people across many different backgrounds, experiences and perspectives. Llama 3.1 addresses users and their needs as they are, without insertion unnecessary judgment or normativity, while reflecting the understanding that even content that may appear problematic in some cases can serve valuable purposes in others. It respects the dignity and autonomy of all users, especially in terms of the values of free thought and expression that power innovation and progress.

But Llama 3.1 is a new technology, and like any new technology, there are risks associated with its use. Testing conducted to date has not covered, nor could it cover, all scenarios. For these reasons, as with all LLMs, Llama 3.1’s potential outputs cannot be predicted in advance, and the model may in some instances produce inaccurate, biased or other objectionable responses to user prompts. Therefore, before deploying any applications of Llama 3.1 models, developers should perform safety testing and tuning tailored to their specific applications of the model. Please refer to available resources including our Responsible Use Guide, Trust and Safety solutions, and other resources to learn more about responsible development. 
""".strip()

res = model.extract_entities(
    text,
    entities = {
        'name'    : 'the model name',
        'creator' : 'the name of the model creator',
        'release' : 'the model release date',
        'licence'      : 'the model licence',
        'tools'        : '(bool) whether the model can use tools or not',
        'code'         : '(bool) whether the model has been trained to code',
        'multimodal'   : '(bool) whether the model can handle multiple data type (like images or audio)',
        'multilingual' : '(bool) whether the model is multilingual or not',
        'languages'    : 'the (list of) languages supported',
        '# parameters' : 'the model size (number parameters)',
        'memory'  : 'the model memory consuption (in Gb). Note that 1B parameters rougly consumes 2Gb of memory.',
    },
    lang = 'en',
    max_input_len = 2048 * 3,
    max_new_tokens = 2048
)[0]

print('# input tokens : {}\nAnswer :\n{}'.format(
    len(res['input_tokens']), res['content']
))

181 tokens generated in 4.348 sec (41.630 tokens/sec)
# input tokens : 5631
Answer :
Here is the requested information :

```json
{
    "name": "Llama 3.1",
    "creator": "Meta",
    "release": "July 23, 2024",
    "licence": "Custom commercial license (Llama 3.1 Community License)",
    "tools": true,
    "code": true,
    "multimodal": false,
    "multilingual": true,
    "languages": ["English", "German", "French", "Italian", "Portuguese", "Hindi", "Spanish", "Thai"],
    "# parameters": ["8B", "70B", "405B"],
    "memory": ["16Gb", "32Gb", "64Gb"]
}
```

Note that the memory consumption is an estimate based on the model size (number of parameters) and the general rule of thumb that 1B parameters roughly consumes 2Gb of memory.


### Retriever Augmented Generator (RAG)

A `Retriever Augmented Generator (RAG)` system is an extension of basic question-answering that enhances the prompt by adding additional information retrieved by a `retriever` model (such as an embedding model). To perform this, the `rag` methods accepts some additional arguments allowing to control the information retrieval pipeline :
- `documents`  : (list of) possibly formatted) files to parse, or parsed documents (`list` of `dict` with (at least) the `text` entry)
- `search_on_web` (**experimental**) : whether to perform web-based search to find relevant information. **Use this feature carefully, as no security check is performed !**
- `retriever` : the retriever name (in this case, the `BAAI/bge-m3` model).
- `retriever_config` : additional kwargs passed to the `retriever.embed` method
- `k` (default `10`) : the number of paragraphs to give to the `LLM`
- `reverse` (default `True`) : whether to pass the retrieved information in `best last` (default) or `best first`. Some recent papers tend to show that passing paragraphs in the decreasing order of score (i.e., the one with highest scores at last), tend to improve answer quality.

This pipeline is currently a basic implementation of a RAG system, and will be improved in future versions. There exist multiple variants / extensions allowing to enhance this pipeline to improve answer quality (such as `query reformulation`, `ArgenticRAG`, ...). 


In [6]:
keras.mixed_precision.set_global_policy('float16')

text = "How does the SV2TTS work ?"

res = model.rag(
    text,
    max_input_len = 4096,
    max_new_tokens = 512,
    
    retriever = 'bge-m3',
    documents = '../**/*.md',
    retriever_config = {
        'batch_size'  : 4,
        'chunk_size'  : 256,
        'group_by'    : 'filename',
        'primary_key' : ('filename', 'text'),
        'use_xla' : True,
        'save' : False,
        'tqdm' : tqdm
    }
)[0]

print('# input tokens : {}\nInput :\n{}\n{}\n\nAnswer :\n{}'.format(
    len(res['input_tokens']), res['input'], "=" * 50, res['content']
))


00%|███████████████████████████████████████████████████████████████████████████████████| 42/42 [00:00<00:00, 58.89it/s]

316 tokens generated in 5.895 sec (53.609 tokens/sec)
# input tokens : 1848
Input :
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 31 December 2024

You are an AI assistant. You have to answer the best as possible to the user's queries<|start_header_id|>user<|end_header_id|>

Here are some sources of information :

<|start_header_id|>document<|end_header_id|>

File :../text_to_speech/README.md



## Notes and references

- Generalized End-to-End loss for Speaker Verification : the GE2E Loss paper (used for speaker encoder in the SV2TTS architecture)

<|start_header_id|>document<|end_header_id|>


 CommonVoice   | `SV2TTSTacotron2`   | `WaveGlow`    | Google Drive | me  | Google Drive  |
| sv2tts_siwis  | `fr`      | SIWIS,  VoxForge,  CommonVoice   | `SV2TTSTacotron2`   | `WaveGlow`    | Google Drive | me  | Google Drive  |
| sv2tts_tacotron2_256_v2   | `fr`      | SIWIS,  VoxForge,  CommonVoice   | `SV2TTSTacotron2`   | `

In [7]:
keras.mixed_precision.set_global_policy('float16')

text = "Comment fonctionne SV2TTS ?"

res = model.rag(
    text,
    lang = 'fr',
    max_input_len = 4096,
    max_new_tokens = 512,
    
    retriever = 'bge-m3',
    documents = '../**/*.md',
    retriever_config = {
        'batch_size'  : 4,
        'chunk_size'  : 256,
        'group_by'    : 'filename',
        'primary_key' : ('filename', 'text'),
        'use_xla' : True,
        'save' : False,
        'tqdm' : tqdm
    }
)[0]

print('# input tokens : {}\nInput :\n{}\n{}\n\nAnswer :\n{}'.format(
    len(res['input_tokens']), res['input'], "=" * 50, res['content']
))


00%|███████████████████████████████████████████████████████████████████████████████████| 42/42 [00:00<00:00, 58.93it/s]

231 tokens generated in 4.405 sec (52.442 tokens/sec)
# input tokens : 1822
Input :
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 31 December 2024

Tu es un assistant IA. Tu dois répondre de manière brève et précise aux questions de l'utilisateur.<|start_header_id|>user<|end_header_id|>

Voici différentes sources d'informations :

<|start_header_id|>document<|end_header_id|>
Fichier :../text_to_speech/README.md



## Notes and references

- Generalized End-to-End loss for Speaker Verification : the GE2E Loss paper (used for speaker encoder in the SV2TTS architecture)


 CommonVoice   | `SV2TTSTacotron2`   | `WaveGlow`    | Google Drive | me  | Google Drive  |
| sv2tts_siwis  | `fr`      | SIWIS,  VoxForge,  CommonVoice   | `SV2TTSTacotron2`   | `WaveGlow`    | Google Drive | me  | Google Drive  |
| sv2tts_tacotron2_256_v2   | `fr`      | SIWIS,  VoxForge,  CommonVoice   | `SV2TTSTacotron2`   | `WaveGlow`    | Google Drive

## Downloading model checkpoint

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = 'meta-llama/Meta-Llama-3.1-8B-Instruct'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model     = AutoModelForCausalLM.from_pretrained(model_name, device_map = 'cuda', torch_dtype = 'float16')

In [ ]:
out = model.generate(tokenizer.encode('Hello World :', return_tensors  = 'pt').cuda())
print(tokenizer.decode(out[0]))